In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0qdj670c
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0qdj670c
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=724c31d485b0f333bb6a8c3cd1830ace57d9a51a5c768fd6925165aad5b1a9b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ub68kaxq/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__device__ bool result;

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weights, bool *currentIS, bool *X) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!X[neighID] && !col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		currentIS[idx] = true;
    X[idx] = true;
    for (uint j = 0; j < deg; j++) {
        uint neighID = str->neighs[offset + j];
        X[neighID] = true;
    }
	}
}

__global__ void checkX(GraphStruct *str, bool* X) {
		uint idx = threadIdx.x + blockDim.x * blockIdx.x;
		if (idx >= str->nodeSize)
				return;

    if(X[idx] == false)
      result = true;
}

__global__ void colorer(Coloring* col, GraphStruct *str, bool *currentIS){
  uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

  if(currentIS[idx])
      col->coloring[idx] = col->numOfColors;
	else
	 		col->uncoloredNodes = true;
}

Coloring* graphColoring(GraphStruct *str){
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);

	bool result_h = true;
	cudaMemcpyToSymbol(result, &result_h, sizeof(bool));

	bool* currentIS;
	cudaMalloc((void**) &currentIS, n * sizeof(bool));
	cudaMemset(currentIS, false, n);

	bool* X;
	cudaMalloc((void**) &X, n * sizeof(bool));
	cudaMemset(X, false, n);
	col->numOfColors = 0;

	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		result_h = true;
		cudaMemset(currentIS, false, n);
		cudaMemset(X, false, n);

    while(result_h){
				result_h = false;
        findIS <<< blocks, threads >>> (col, str, weigths, currentIS, X);

		    //cudaDeviceSynchronize();
				checkX <<< blocks, threads >>> (str, X);
    }

    colorer <<< blocks, threads >>> (col, str, currentIS);
		cudaDeviceSynchronize();
	}

  cudaFree(states);
	cudaFree(weigths);
  return col;
}

__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx > n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}


'File written in /content/src/MIS.cu'

In [ ]:
%%cuda --name test_MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"

using namespace std;

int main(void) {
	unsigned int n = 1000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	for(int i = 0; i < n; i++){
			printf("%d ", col->coloring[i]);
	}

	//printColoring(col, str, 1);



	return EXIT_SUCCESS;
}

'File written in /content/src/test_MIS.cu'

In [ ]:
!nvcc -dc -D__CDPRT_SUPPRESS_SYNC_DEPRECATION_WARNING src/test_MIS.cu /content/src/MIS.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_MIS.o MIS.o graph.o graph_d.o -o test_MIS
!./test_MIS

230.100998 ms
82 457 148 4 274 493 9 171 283 552 48 109 545 153 126 259 439 209 406 338 186 339 521 376 412 69 145 128 153 422 131 234 89 533 351 418 522 134 58 38 342 86 99 368 556 66 343 159 463 361 199 519 44 73 460 185 136 468 418 57 370 96 471 45 442 175 429 366 329 473 567 85 40 478 426 535 317 499 557 247 113 76 122 168 331 284 501 69 485 408 111 546 101 231 514 158 434 326 364 173 343 205 199 121 292 266 109 61 98 396 349 451 462 383 338 183 389 102 65 558 398 70 555 550 144 492 338 117 371 560 433 275 100 143 541 52 548 195 563 382 339 355 232 154 17 403 375 416 135 186 214 240 46 350 504 308 68 423 238 233 213 245 54 145 220 228 103 549 139 185 41 312 461 330 16 140 449 97 479 423 149 437 132 19 383 115 544 347 391 228 547 413 490 552 25 377 95 12 470 249 390 244 26 227 258 2 219 188 64 250 240 400 37 314 59 374 120 484 195 447 554 490 251 114 526 109 476 76 257 296 17 487 92 393 456 42 199 127 6 418 87 362 198 506 353 60 217 443 541 516 518 207 187 38 438 467 347 393 389 441